In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from IPython.display import clear_output
!pip install peft==0.8.2
!pip install bitsandbytes==0.42.0
!pip install accelerate==0.34.2
!pip install datasets==2.16.1
!pip install GPUtil
!pip install transformers==4.43.1
clear_output()

In [ ]:
import warnings
import os
from transformers import set_seed

SEED = 123
set_seed(SEED)

warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

INPUT_DIR = "/kaggle/input/augclaimswithrag1/"
DIR = '/kaggle/working/'

NUM_WORKERS = os.cpu_count()
NUM_CLASSES = 3

EPOCHS,R,LORA_ALPHA,LORA_DROPOUT = 5,64,32,0.1
BATCH_SIZE = 4

MODEL_ID = '/kaggle/input/llama-3.2/transformers/1b/1'
# MODEL_ID = '/kaggle/input/llama-3.2/transformers/3b/1'

## Load the Dataset

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the CSV into a DataFrame
df = pd.read_csv(f'{INPUT_DIR}AugClaimWithRAG1.csv')

# Drop rows with missing claims or labels, if necessary
df = df.dropna(subset=['claim', 'label'])

# Convert the 'label' column to integers
df['label'] = df['label'].astype(int)

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into train and test sets
dataset = dataset.train_test_split(test_size=0.2)

print(dataset)

# Print a sample to verify
print(dataset["train"][0], dataset['test'][0])


## Baseline Accuracy

In [ ]:
from collections import Counter

train_len, test_len = len(dataset['train']), len(dataset['test'])

train_dataset_label_counts = Counter(dataset['train']['label'])
test_dataset_label_counts = Counter(dataset['test']['label'])

print(f"Train dataset: {train_len} samples, {train_dataset_label_counts}")
print(f"Test dataset: {test_len} samples, {test_dataset_label_counts}")

test_majority_class = test_dataset_label_counts.most_common(1)[0]

baseline_accuracy = test_majority_class[1] / test_len

print(f"Baseline accuracy: {baseline_accuracy:.2%}")

## Tokenization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
print(tokenizer.padding_side, tokenizer.pad_token)
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.padding_side, tokenizer.pad_token)
tokenizer.add_special_tokens({'additional_special_tokens': ['[SEP]']})

In [ ]:
tokenized_dataset = {}

# Tokenize the claim and abstract
def tokenize_function(examples):
    claims_and_abstracts = [
        claim + tokenizer.additional_special_tokens[0] + abstract  # Using eos_token as separator
        for claim, abstract in zip(examples["claim"], examples["abstract"])
    ]
    
    # Tokenize the combined claim and abstract text
    return tokenizer(
        claims_and_abstracts,
        truncation=True,
        padding="max_length",
        max_length=512  # Adjust max length as needed
    )

# Apply the tokenizer to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)


## Load and set up the model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=3,
    load_in_8bit=True,
)
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
print(model)

### Vanilla Model to LoRA Model

In [ ]:
from peft import prepare_model_for_int8_training, LoraConfig, TaskType, get_peft_model

# Prepare model for int8 training and configure LoRA
model = prepare_model_for_int8_training(model)

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(
    r=R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    task_type=TaskType.SEQ_CLS,
    target_modules='all-linear'
)
lora_config

In [ ]:
lora_model = get_peft_model(model, lora_config)

In [ ]:
lora_model.print_trainable_parameters()


## Train the Model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import os
from transformers import TrainerCallback

# Callback to store metrics
class MetricCallback(TrainerCallback):
    def __init__(self):
        self.train_loss = []
        self.val_loss = []
        self.train_accuracy = []
        self.val_accuracy = []
        self.train_precision = []
        self.val_precision = []
        self.train_recall = []
        self.val_recall = []
        self.train_f1 = []
        self.val_f1 = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            # Store the logs for loss and accuracy for both training and validation
            if 'loss' in logs:
                self.train_loss.append(logs['loss'])
            if 'eval_loss' in logs:
                self.val_loss.append(logs['eval_loss'])
            if 'eval_accuracy' in logs:
                self.val_accuracy.append(logs['eval_accuracy'])
            if 'accuracy' in logs:
                self.train_accuracy.append(logs['accuracy'])
            if 'eval_precision' in logs:
                self.val_precision.append(logs['eval_precision'])
            if 'eval_recall' in logs:
                self.val_recall.append(logs['eval_recall'])
            if 'eval_f1' in logs:
                self.val_f1.append(logs['eval_f1'])
            if 'precision' in logs:
                self.train_precision.append(logs['precision'])
            if 'recall' in logs:
                self.train_recall.append(logs['recall'])
            if 'f1' in logs:
                self.train_f1.append(logs['f1'])

    def on_step_end(self, args, state, control, logs=None, **kwargs):
        # Capture training metrics at the end of a step
        if logs is not None and 'loss' in logs:
            self.train_loss.append(logs['loss'])

# Initialize callback
metric_callback = MetricCallback()

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define metric computation function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    # Calculate precision, recall, and F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    
    accuracy = (predictions == labels).mean()
    
    # Return metrics as a dictionary
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Set up the Trainer with training arguments
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/",
        learning_rate=2e-5,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=20,
        weight_decay=0.01,
        load_best_model_at_end=True,
        logging_steps=10,
        report_to="none"
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[metric_callback]
)


In [ ]:
print("Training the Model")
trainer.train()

In [ ]:
# Plotting function
def plot_acc():
    epochs = range(1, len(metric_callback.val_accuracy) + 1)

    # Plot Validation Accuracy
    plt.figure(figsize=(6, 6))
    # plt.subplot(2, 2, 1)
    # plt.plot(epochs, metric_callback.train_accuracy, label='Train Accuracy')
    plt.plot(epochs, metric_callback.val_accuracy, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    # plt.legend()
    plt.grid(True)
    plt.title('Validation Accuracy')
    plt.savefig("LLAMA1B_AUG_accuracy.png")
    plt.show()
    
    
def plot_loss():
    # Plot Validation Loss
    epochs = range(1, len(metric_callback.val_accuracy) + 1)
    plt.figure(figsize=(6, 6))
    # plt.plot(epochs, metric_callback.train_loss, label='Train Loss')
    plt.plot(epochs, metric_callback.val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    # plt.legend()
    plt.grid(True)
    plt.title('Validation Loss')
    plt.savefig("LLAMA1B_AUG_loss.png")
    plt.show()
    
def plot_prec():
    # Plot Validation Precision
    epochs = range(1, len(metric_callback.val_precision) + 1)
    plt.figure(figsize=(6, 6))
    # plt.plot(epochs, metric_callback.train_precision, label='Train Precision')
    plt.plot(epochs, metric_callback.val_precision, label='Validation Precision')
    plt.xlabel('Epochs')
    plt.ylabel('Precision')
    # plt.legend()
    plt.grid(True)
    plt.title('Validation Precision')
    plt.savefig("LLAMA1B_AUG_precision.png")
    plt.show()
    
def plot_recall():
    # Plot Validation Recall
    epochs = range(1, len(metric_callback.val_recall) + 1)
    plt.figure(figsize=(6, 6))
    # plt.plot(epochs, metric_callback.train_recall, label='Train Recall')
    plt.plot(epochs, metric_callback.val_recall, label='Validation Recall')
    plt.xlabel('Epochs')
    plt.ylabel('Recall')
    # plt.legend()
    plt.grid(True)
    plt.title('Validation Recall')
    plt.savefig("LLAMA1B_AUG_recall.png")
    plt.show()
    
def plot_F1():
    # Plot Validation F1
    epochs = range(1, len(metric_callback.val_f1) + 1)
    plt.figure(figsize=(6, 6))
    # plt.plot(epochs, metric_callback.train_f1, label='Train F1')
    plt.plot(epochs, metric_callback.val_f1, label='Validation F1')
    plt.xlabel('Epochs')
    plt.ylabel('F1-Score')
    # plt.legend()
    plt.grid(True)
    plt.title('Validation F1-Score')
    plt.savefig("LLAMA1B_AUG_f1.png")
    plt.show()


# Call the plot function after training
plot_acc()
plot_loss()
plot_prec()
plot_recall()
plot_F1()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Function to generate and plot confusion matrix
def plot_confusion_matrix_from_trainer(trainer, test_dataset, labels):
    # Evaluate the model
    predictions, labels_list, _ = trainer.predict(test_dataset)
    predicted_labels = np.argmax(predictions, axis=1)
    
    # Generate confusion matrix
    cm = confusion_matrix(labels_list, predicted_labels, labels=range(len(labels)))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    
    # Plot the confusion matrix
    plt.figure(figsize=(8, 8))
    disp.plot(cmap=plt.cm.Blues, values_format='d', ax=plt.gca())
    plt.title("Confusion Matrix")
    plt.savefig("LLAMA1B_AUG_CM.png")
    plt.show()
    

# Usage
print("Evaluating the Model and Plotting Confusion Matrix")
plot_confusion_matrix_from_trainer(
    trainer=trainer,
    test_dataset=tokenized_dataset["test"],
    labels=["0", "1", "2"]  # Replace with your class names
)
